# Форматы данных (1)

Материалы:
* Макрушин С.В. "Лекция 4: Форматы данных"
* https://docs.python.org/3/library/json.html
* https://docs.python.org/3/library/pickle.html
* https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
* Уэс Маккини. Python и анализ данных

## Лабораторная работа №4

### JSON

In [11]:
import json
import pandas as pd

1.1 Считайте файл `contributors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [12]:
with open('contributors_sample.json') as f:
    content = f.read()
    
contributors = json.loads(content)
print(contributors[:3])

[{'username': 'uhebert', 'name': 'Lindsey Nguyen', 'sex': 'F', 'address': '01261 Cameron Spring\nTaylorfurt, AK 97791', 'mail': 'jsalazar@gmail.com', 'jobs': ['Energy engineer', 'Engineer, site', 'Environmental health practitioner', 'Biomedical scientist', 'Jewellery designer'], 'id': 35193}, {'username': 'vickitaylor', 'name': 'Cheryl Lewis', 'sex': 'F', 'address': '66992 Welch Brooks\nMarshallshire, ID 56004', 'mail': 'bhudson@gmail.com', 'jobs': ['Music therapist', 'Volunteer coordinator', 'Designer, interior/spatial'], 'id': 91970}, {'username': 'sheilaadams', 'name': 'Julia Allen', 'sex': 'F', 'address': 'Unit 1632 Box 2971\nDPO AE 23297', 'mail': 'darren44@yahoo.com', 'jobs': ['Management consultant', 'Engineer, structural', 'Lecturer, higher education', 'Theatre manager', 'Designer, textile'], 'id': 1848091}]


1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

In [13]:
print(set([c['address'].split()[0] for c in contributors]))

{'9237', '46361', '8020', '3864', '4495', '352', '603', '736', '17660', '6643', '847', '67871', '8786', '797', '41362', '65979', '87064', '9334', '48859', '46207', '48724', '404', '59347', '31148', '8622', '182', '4954', '16178', '34627', '26851', '87793', '40111', '4651', '59658', '1202', '4108', '4309', '37727', '7261', '39652', '38659', '9106', '83108', '1789', '8978', '90390', '1062', '807', '36781', '57485', '80638', '82145', '512', '53076', '14111', '0168', '026', '80046', '88791', '80574', '22181', '6084', '100', '904', '055', '563', '972', '0185', '860', '94566', '95026', '28927', '04405', '1114', '7587', '28984', '55774', '2415', '91171', '88230', '9953', '32153', '0988', '9212', '55396', '3319', '304', '4218', '8408', '24507', '622', '07564', '670', '6753', '2626', '84055', '9973', '263', '7954', '676', '1862', '9668', '11087', '029', '60540', '71065', '9405', '17080', '80742', '893', '389', '5986', '880', '2527', '173', '6842', '5810', '2940', '34915', '53275', '9498', '907'

1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [14]:
def find_by_username(username: str):
    for contributor in contributors:
        if contributor['username'] == username:
            return contributor
    raise ValueError("Contributor doesn't exist")

print(find_by_username('uhebert'))    # ищем существующего пользователя
print(find_by_username('kfjkajl'))    # ищем несуществующего пользователя

{'username': 'uhebert', 'name': 'Lindsey Nguyen', 'sex': 'F', 'address': '01261 Cameron Spring\nTaylorfurt, AK 97791', 'mail': 'jsalazar@gmail.com', 'jobs': ['Energy engineer', 'Engineer, site', 'Environmental health practitioner', 'Biomedical scientist', 'Jewellery designer'], 'id': 35193}


ValueError: Contributor doesn't exist

1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

In [15]:
print(f'Количество женщин: {len([c for c in contributors if c["sex"] == "F"])}')
print(f'Количество мужчин: {len([c for c in contributors if c["sex"] == "M"])}')

Количество женщин: 2136
Количество мужчин: 2064


1.5 Создайте `pd.DataFrame` `contributors`, имеющий столбцы `id`, `username` и `sex`.

In [16]:
pd_contributors = pd.read_json('contributors_sample.json').drop(['name', 'address', 'mail', 'jobs'], axis=1)
pd_contributors

,username,sex,id
0,uhebert,F,35193
1,vickitaylor,F,91970
2,sheilaadams,F,1848091
3,nicole82,F,50969
4,jean67,M,676820
...,...,...,...
4195,stevenspencer,F,423555
4196,rwilliams,M,35251
4197,lmartinez,F,135887
4198,brendahill,M,212714


1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `contributors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует? 

In [17]:
recipes = pd.read_csv('recipes_sample.csv')
contributors_and_recipes = pd.merge(
                                recipes,
                                pd_contributors,
                                left_on='contributor_id',
                                right_on='id',
                                how='left'
                            ).drop(['id_x', 'id_y'], axis=1)

contributors_and_recipes[contributors_and_recipes['username'].isna()].shape[0]

15059

### pickle

2.1 На основе файла `contributors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

In [18]:
job_people = {}
for contributor in contributors:
    for job in contributor['jobs']:
        job_people.setdefault(job, []).append(contributor['username'])
print(job_people)

{'Energy engineer': ['uhebert', 'annmoore', 'garysilva', 'martinezashley', 'sextonsheila', 'pjames', 'smithjonathan', 'wardjames', 'cwheeler', 'ucarlson', 'robert71', 'johnsontheresa', 'amanda41', 'stacey47', 'timothynelson', 'timothynelson', 'rogersmichael', 'melissa94', 'wmcdaniel', 'charles74', 'smithjennifer', 'clintonjones'], 'Engineer, site': ['uhebert', 'nancy12', 'andrea03', 'catherineross', 'wesley32', 'natalieross', 'rossdoris', 'christophersmith', 'dbooker', 'ericarobertson', 'trantricia', 'tpugh', 'jasonvelez', 'samantha36', 'brandidaniels', 'tenglish', 'reyesbrett', 'austin18', 'vjohnson', 'zmejia', 'daniel04', 'cynthia20', 'morgan15', 'avaldez', 'jessica92', 'laurieholloway', 'baileyvictoria'], 'Environmental health practitioner': ['uhebert', 'jonathanchristian', 'xjohnson', 'dsmith', 'james01', 'nancytaylor', 'ztaylor', 'andrewwoods', 'susan54', 'fmaldonado', 'james74', 'bakerjacob', 'stephanie81', 'whitejoseph', 'qolson', 'hknox', 'gonzalesdaniel', 'tranronald', 'jesseg

2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

In [21]:
import pickle
import os

with open('job_people.pickle', 'wb') as p_file:
    pickle.dump(job_people, p_file)
    
with open('job_people.json', 'w') as j_file:
    json.dump(job_people, j_file, indent=4)
    
print(os.stat('job_people.pickle').st_size)
print(os.stat('job_people.json').st_size)

132272
389655


2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

In [22]:
with open('job_people.pickle', 'rb') as pc_file:
    print(pickle.load(pc_file))

{'Energy engineer': ['uhebert', 'annmoore', 'garysilva', 'martinezashley', 'sextonsheila', 'pjames', 'smithjonathan', 'wardjames', 'cwheeler', 'ucarlson', 'robert71', 'johnsontheresa', 'amanda41', 'stacey47', 'timothynelson', 'timothynelson', 'rogersmichael', 'melissa94', 'wmcdaniel', 'charles74', 'smithjennifer', 'clintonjones'], 'Engineer, site': ['uhebert', 'nancy12', 'andrea03', 'catherineross', 'wesley32', 'natalieross', 'rossdoris', 'christophersmith', 'dbooker', 'ericarobertson', 'trantricia', 'tpugh', 'jasonvelez', 'samantha36', 'brandidaniels', 'tenglish', 'reyesbrett', 'austin18', 'vjohnson', 'zmejia', 'daniel04', 'cynthia20', 'morgan15', 'avaldez', 'jessica92', 'laurieholloway', 'baileyvictoria'], 'Environmental health practitioner': ['uhebert', 'jonathanchristian', 'xjohnson', 'dsmith', 'james01', 'nancytaylor', 'ztaylor', 'andrewwoods', 'susan54', 'fmaldonado', 'james74', 'bakerjacob', 'stephanie81', 'whitejoseph', 'qolson', 'hknox', 'gonzalesdaniel', 'tranronald', 'jesseg

### XML

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`